## Install libraries and set up keys

In [5]:
%%capture

# Note: This could take up to 5 minutes

!pip install langchain              # Base LangChain
!pip install langchainhub           # Hub for prompt templates
!pip install langchain-core         # Core LangChain components
!pip install langchain-community    # Extenteded list of LangChain components
!pip install langchain-anthropic    # LangChain bindings for Anthropic LLM
!pip install langchain-openai       # LangChain bindings for OpenAI LLM
!pip install google-search-results  # Serp API
!pip install pypdf                  # Parse PDF docs
# !pip install sentence_transformers  # Embedding alternative to OpenAI embedding
!pip install chromadb               # Vector DB

In [6]:
import os
from google.colab import userdata

os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
# os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Load PDFs in Vector DB

Fetch the PDFs that we will load in the Vector DB from Github.

In [7]:
%%capture

!rm -rf ./repo
!git clone --depth 1 https://github.com/adeshmukh/gaiip-intro-orchestration.git ./repo

Create Chroma Documents from the PDFs. Use splitter to make smaller chunks.


In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import logging

# Ignore pypdf warnings
logging.getLogger("pypdf").setLevel(logging.ERROR)


base_path = "./repo/pdfs"
pdf_documents = os.listdir(base_path)
docs = []

# Load docs as Chroma Document objects
for pdf_document in pdf_documents:
  pdf_loader = PyPDFLoader(f"{base_path}/{pdf_document}")
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
  pages = pdf_loader.load_and_split(text_splitter=splitter)
  docs.extend(pages)
  print(f"Loaded {len(pages)} pages from {pdf_document}")


Loaded 31 pages from us-national-budget-health-fy24.pdf
Loaded 6 pages from us-national-budget-nasa-fy24.pdf
Loaded 7 pages from us-national-budget-civil-fy24.pdf
Loaded 7 pages from us-national-budget-nsf-fy24.pdf
Loaded 7 pages from us-national-budget-treasury-fy24.pdf
Loaded 6 pages from us-national-budget-ssa-fy24.pdf


Load the Chroma Documents into Chroma DB

In [9]:
%%capture

from langchain.tools.retriever import create_retriever_tool
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize the vector DB with the docs
vector_db = Chroma.from_documents(docs, embedding_function)

# Maximal Marginal Relevance (MMR) optimizes for similarity to query and diversity among selected documents.
retriever = vector_db.as_retriever(search_type='mmr')

# Wrap the retriever in a tool
tool = create_retriever_tool(
    retriever,
    "us_fiscal_budget_fy2024",
    "Searches and returns excerpts from the US national budget for the fiscal year 2024.",
)

[Optional] Test searching in the Chroma DB

In [10]:
# Try querying the retriever to see how it functions
hits = retriever.invoke("What is the total requested budget for NASA and NSF?")

for hit in hits[:10]: # Peek at the top 10 hits
  print('-------------------------------------------')
  print(hit.page_content, hit.metadata)


-------------------------------------------
127NATIONAL SCIENCE FOUNDATIONThe National Science Foundation (NSF) is responsible for science education and promoting the 
progress of science and innovation.  The President’s 2024 Budget for NSF advances the goals 
of the CHIPS and Science Act by:  strengthening U.S. leadership in emerging technologies; 
expanding the science, technology, engineering, and mathematics (STEM) workforce while 
advancing equity; boosting research and development (R&D) including research infrastructure; 
and combating the climate crisis. 
The Budget requests $11.3 billion in discretionary budget authority for 2024, a $1.8 billion or 
18.6-percent increase from the 2023 enacted level to support the CHIPS and Science Act.
The President’s 2024 Budget:  
• Ensures the Future Is Made in America.   NSF plays a key role in the CHIPS and Science 
Act with its focus on U.S. leadership in new technologies—from artiﬁcial intelligence to {'page': 0, 'source': './repo/pdfs/u

## Zero-shot ReAct agent

Here we build a zero-shot ReAct agent with LLM and 2 tools:
1. serpapi: This provides the agent with web search capability
2. llm-math: This provides the agent with ability to perform math calculations

In [13]:
from langchain import hub
from langchain.agents import (
    create_react_agent,
    load_tools,
    AgentType,
    AgentExecutor,
  )
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0) # temperature = 0 results in fewer hallucinations

# Uncomment below lines to use Anthropic LLM instead of OpenAI
# from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model='claude-3-opus-20240229')

# Tools that agents can use to augment their capabilities
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Initialize agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools)


Let's put the agent to test

In [14]:
agent_executor.invoke({"input": "What is the total requested budget for NASA and NSF for Fiscal Year 2024?"}, config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the total requested budget for NASA and NSF for Fiscal Year 2024?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] E

{'input': 'What is the total requested budget for NASA and NSF for Fiscal Year 2024?',
 'output': '900.107 billion dollars'}

Let's try another test.

In [15]:
agent_executor.invoke({"input": "Which of the following cities has the least forecasted temperature on June 20th: Chicago, Houston, Jacksonville?"})

{'input': 'Which of the following cities has the least forecasted temperature on June 20th: Chicago, Houston, Jacksonville?',
 'output': 'Chicago has the least forecasted temperature on June 20th.'}

In [16]:
agent_executor.invoke({"input": "Which of the following cities has the least forecasted temperature on June 20th: Chicago, Houston, Jacksonville?"},
                      config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Which of the following cities has the least forecasted temperature on June 20th: Chicago, Houston, Jacksonville?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_s

{'input': 'Which of the following cities has the least forecasted temperature on June 20th: Chicago, Houston, Jacksonville?',
 'output': 'Chicago has the least forecasted temperature on June 20th.'}